# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

Below downloading links are expired, use the links above instead.

In [24]:
# # !curl 
# !export http_proxy=http://proxy.hkg.sap.corp:8080
# !export https_proxy=http://proxy.hkg.sap.corp:8080
# # !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partaa
# # !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partab
# # !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partac
# # !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partad

# !curl -O https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partaa
# !curl -O https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partab
# !curl -O https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partac
# !curl -O https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partad

# !cat Dataset.tar.gz.part* > Dataset.tar.gz

# # unzip the file
# !tar zxvf Dataset.tar.gz

## Fix Random Seed

In [25]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [26]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class MyDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len
    
        # Load the mapping from speaker neme to their corresponding id. 
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]
    
        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]
    
        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
    def __len__(self):
            return len(self.data)
 
    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # Segmemt mel-spectrogram into "segment_len" frames.
        if len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker
 
    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [27]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = MyDataset(data_dir)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1, nhead=8):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100

        # d_model 词向量维度
        # dim_ff ff层维度
        # nhead 多头注意力数量
        # dropout 
        # activation 激活函数，relu
        # layer_norm_eps 在BatchN时，在分母里面加上这个值，防止方差过小除以0
        # batch_first input和output使用形如(batch, seq, feature)，把batch放在前面
        # norm_first norm是先于还是后于 attention和ff，默认false
        # bias 如果true，线性层和norm层不会学习bias
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, dim_feedforward=256, nhead=8
        )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, 2*d_model),
            nn.Sigmoid(),
            nn.Dropout(dropout),
            nn.Linear(2*d_model, n_spks),
        )

    def forward(self, mels):
        """
        args:
            mels: (batch size, length, 40)
        return:
            out: (batch size, n_spks)
        """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        # mean pooling
        stats = out.mean(dim=1)

        # out: (batch, n_spks)
        out = self.pred_layer(stats)
        return out

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [29]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
    """
    Create a schedule with a learning rate that decreases following the values of the cosine function between the
    initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
    initial lr set in the optimizer.

    Args:
        optimizer (:class:`~torch.optim.Optimizer`):
        The optimizer for which to schedule the learning rate.
        num_warmup_steps (:obj:`int`):
        The number of steps for the warmup phase.
        num_training_steps (:obj:`int`):
        The total number of training steps.
        num_cycles (:obj:`float`, `optional`, defaults to 0.5):
        The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
        following a half-cosine).
        last_epoch (:obj:`int`, `optional`, defaults to -1):
        The index of the last epoch when resuming training.

    Return:
        :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return max(
            0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [30]:
import torch


def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)

    loss = criterion(outs, labels)

    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [31]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)

# Main function

In [32]:
def parse_args(isTrain=True):
    shared_config = {
        "data_dir": "./Dataset",
        "model_save_path": "model.ckpt",
        "d_model": 512,
        "nhead": 16
    }

    training_config = {  # Corrected spelling here
        "batch_size": 8,
        "n_workers": 0,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 70000
    }

    validation_config = {
        "output_path": "./output.csv",
    }

    if isTrain:
        return {**shared_config, **training_config}  # Corrected spelling here
    else:
        return {**shared_config, **validation_config}


In [33]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def main(
    data_dir,
    model_save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
    d_model,
    nhead
    
):
    """Main function."""
    device = torch.device('mps') if torch.device('mps') else torch.device('cpu')
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    model = Classifier(n_spks=speaker_num, d_model=d_model,nhead = nhead).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit="step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, model_save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args(True))

[Info]: Use mps now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [03:00<00:00, 11.08 step/s, accuracy=0.00, loss=5.95, step=2000]
Valid: 100% 5664/5667 [00:44<00:00, 126.86 uttr/s, accuracy=0.00, loss=6.02]
Train: 100% 2000/2000 [02:25<00:00, 13.73step/s, accuracy=0.00, loss=5.63, step=4000]
Valid: 100% 5664/5667 [00:35<00:00, 157.58 uttr/s, accuracy=0.00, loss=5.91]
Train: 100% 2000/2000 [02:27<00:00, 13.59step/s, accuracy=0.00, loss=5.75, step=6000]
Valid: 100% 5664/5667 [00:36<00:00, 155.67 uttr/s, accuracy=0.00, loss=5.89]
Train: 100% 2000/2000 [02:05<00:00, 16.00step/s, accuracy=0.00, loss=5.88, step=8000]
Valid: 100% 5664/5667 [00:40<00:00, 140.08 uttr/s, accuracy=0.01, loss=5.86]
Train: 100% 2000/2000 [01:57<00:00, 17.05step/s, accuracy=0.00, loss=5.91, step=1e+4]
Valid: 100% 5664/5667 [00:36<00:00, 153.76 uttr/s, accuracy=0.01, loss=5.77]
Train:   0% 3/2000 [00:00<03:38,  9.14step/s, accuracy=0.00, loss=5.75, step=1e+4]

Step 10000, best model saved. (accuracy=0.0106)


Train: 100% 2000/2000 [02:11<00:00, 15.20step/s, accuracy=0.00, loss=6.01, step=12000]
Valid: 100% 5664/5667 [00:39<00:00, 143.23 uttr/s, accuracy=0.01, loss=5.71]
Train: 100% 2000/2000 [02:10<00:00, 15.32step/s, accuracy=0.00, loss=5.87, step=14000]
Valid: 100% 5664/5667 [00:11<00:00, 498.16 uttr/s, accuracy=0.01, loss=5.70]
Train: 100% 2000/2000 [02:07<00:00, 15.64step/s, accuracy=0.00, loss=5.69, step=16000]
Valid: 100% 5664/5667 [00:18<00:00, 303.43 uttr/s, accuracy=0.02, loss=5.55]
Train: 100% 2000/2000 [02:05<00:00, 15.92step/s, accuracy=0.00, loss=5.55, step=18000]
Valid: 100% 5664/5667 [00:23<00:00, 242.85 uttr/s, accuracy=0.02, loss=5.43]
Train: 100% 2000/2000 [01:54<00:00, 17.42step/s, accuracy=0.00, loss=4.95, step=2e+4] 
Valid: 100% 5664/5667 [00:26<00:00, 213.61 uttr/s, accuracy=0.03, loss=5.32]
Train:   0% 3/2000 [00:00<02:32, 13.07step/s, accuracy=0.00, loss=4.77, step=2e+4]

Step 20000, best model saved. (accuracy=0.0300)


...

Step 50000, best model saved. (accuracy=0.1764)


Train: 100% 2000/2000 [02:16<00:00, 14.65step/s, accuracy=0.50, loss=3.01, step=52000]
Valid: 100% 5664/5667 [00:36<00:00, 156.05 uttr/s, accuracy=0.18, loss=4.01]
Train: 100% 2000/2000 [02:07<00:00, 15.66step/s, accuracy=0.25, loss=3.39, step=54000]
Valid: 100% 5664/5667 [00:34<00:00, 165.18 uttr/s, accuracy=0.20, loss=3.92]
Train: 100% 2000/2000 [02:11<00:00, 15.16step/s, accuracy=0.00, loss=3.94, step=56000]
Valid: 100% 5664/5667 [00:35<00:00, 159.93 uttr/s, accuracy=0.20, loss=3.93]
Train: 100% 2000/2000 [02:03<00:00, 16.16step/s, accuracy=0.12, loss=3.70, step=58000]
Valid: 100% 5664/5667 [00:43<00:00, 130.05 uttr/s, accuracy=0.21, loss=3.87]
Train: 100% 2000/2000 [02:39<00:00, 12.54step/s, accuracy=0.12, loss=3.64, step=6e+4] 
Valid: 100% 5664/5667 [00:24<00:00, 231.09 uttr/s, accuracy=0.20, loss=3.90]
Train:   0% 3/2000 [00:00<02:21, 14.15step/s, accuracy=0.12, loss=4.19, step=6e+4]

Step 60000, best model saved. (accuracy=0.2087)


Train: 100% 2000/2000 [02:18<00:00, 14.45step/s, accuracy=0.12, loss=4.64, step=62000]
Valid: 100% 5664/5667 [00:36<00:00, 157.16 uttr/s, accuracy=0.22, loss=3.86]
Train: 100% 2000/2000 [02:04<00:00, 16.03step/s, accuracy=0.12, loss=3.86, step=64000]
Valid: 100% 5664/5667 [00:33<00:00, 169.10 uttr/s, accuracy=0.21, loss=3.88]
Train: 100% 2000/2000 [02:08<00:00, 15.59step/s, accuracy=0.50, loss=3.17, step=66000]
Valid: 100% 5664/5667 [00:39<00:00, 144.25 uttr/s, accuracy=0.22, loss=3.87]
Train: 100% 2000/2000 [02:08<00:00, 15.61step/s, accuracy=0.25, loss=3.72, step=68000]
Valid: 100% 5664/5667 [00:38<00:00, 146.73 uttr/s, accuracy=0.21, loss=3.84]
Train: 100% 2000/2000 [02:35<00:00, 12.87step/s, accuracy=0.25, loss=3.63, step=7e+4] 
Valid: 100% 5664/5667 [00:21<00:00, 258.51 uttr/s, accuracy=0.21, loss=3.87]
Train:   0% 0/2000 [00:00<?, ?step/s]


Step 70000, best model saved. (accuracy=0.2177)


# Inference

## Dataset of inference

In [34]:
from ds import InferenceDataset, inference_collate_batch

## Main funcrion of Inference

In [23]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader


def main(
    data_dir,
    model_save_path,
    output_path,
    d_model,
    nhead
):
    """Main function."""
    device = torch.device('mps') if torch.device('mps') else torch.device('cpu')
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    print(InferenceDataset)
    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        # num_workers=8,
        collate_fn=inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!",flush = True)

    speaker_num = len(mapping["id2speaker"])
    model = Classifier(n_spks=speaker_num, d_model=d_model, nhead=nhead).to(device)
    model.load_state_dict(torch.load(model_save_path))
    model.eval()
    print(f"[Info]: Finish creating model!",flush = True)

    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            outs = model(mels)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


if __name__ == "__main__":
    main(**parse_args(isTrain=False))

[Info]: Use mps now!
<class 'ds.InferenceDataset'>
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]